In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# used to supress display of warnings
import warnings

from sklearn.metrics import precision_recall_curve,accuracy_score,f1_score,precision_score,recall_score

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# suppress display of warnings
warnings.filterwarnings('ignore')

In [4]:
import os
source_dir=os.path.join('/content','gdrive','MyDrive','105_classes_pins_dataset')

In [37]:
class IdentityMetadata():
    def __init__(self, base, name, file):
        self.base = base
        # identity name
        self.name = name
        # image file name
        self.file = file

    def __repr__(self):
        return self.image_path()

    def image_path(self):
        return os.path.join(self.base, self.name, self.file)

def load_metadata(path):
    metadata = []
    for i in os.listdir(path):
        for f in os.listdir(os.path.join(path, i))[:50]:
            # Check file extension. Allow only jpg/jpeg' files.
            ext = os.path.splitext(f)[1]
            if ext == '.jpg' or ext == '.jpeg':
                metadata.append(IdentityMetadata(path, i, f))
    return np.array(metadata)

# metadata = load_metadata('images')
metadata = load_metadata(source_dir)

In [38]:
print('metadata shape :', metadata.shape)

metadata shape : (5250,)


In [46]:
metadata[150]

/content/gdrive/MyDrive/105_classes_pins_dataset/pins_Tom Holland/Tom Holland104_4692.jpg

In [47]:
type(metadata[1500]), metadata[1500].image_path()

(__main__.IdentityMetadata,
 '/content/gdrive/MyDrive/105_classes_pins_dataset/pins_Megan Fox/Megan Fox101_3368.jpg')

In [48]:
import cv2
def load_image(path):
    img = cv2.imread(path, 1)
    # OpenCV loads images with color channels
    # in BGR order. So we need to reverse them
    return img[...,::-1]

In [49]:
load_image('/content/gdrive/MyDrive/105_classes_pins_dataset/pins_Ursula Corbero/Ursula Corbero244_110.jpg')

array([[[105, 196, 161],
        [106, 193, 158],
        [109, 188, 156],
        ...,
        [ 95, 180, 151],
        [ 90, 175, 146],
        [ 86, 170, 146]],

       [[104, 194, 158],
        [104, 190, 155],
        [108, 185, 151],
        ...,
        [ 90, 175, 144],
        [ 84, 169, 140],
        [ 80, 164, 138]],

       [[102, 186, 150],
        [102, 182, 145],
        [108, 180, 143],
        ...,
        [ 88, 173, 142],
        [ 81, 164, 136],
        [ 76, 159, 133]],

       ...,

       [[177, 101,  69],
        [179, 103,  71],
        [183, 104,  73],
        ...,
        [ 61,  36,  32],
        [ 61,  36,  32],
        [ 61,  36,  32]],

       [[175,  98,  68],
        [178, 101,  71],
        [181, 102,  71],
        ...,
        [ 60,  35,  31],
        [ 61,  36,  32],
        [ 61,  36,  31]],

       [[173,  96,  66],
        [176,  99,  69],
        [179, 100,  69],
        ...,
        [ 60,  35,  31],
        [ 61,  36,  32],
        [ 61,  36,  31]]

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ZeroPadding2D, Convolution2D, MaxPooling2D, Dropout, Flatten, Activation

def vgg_face():
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Convolution2D(4096, (7, 7), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation('softmax'))
    return model

In [12]:
model = vgg_face()

model.load_weights('/content/gdrive/MyDrive/vgg_face_weights.h5')

In [13]:
from tensorflow.keras.models import Model
vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

In [14]:
type(vgg_face_descriptor)

keras.src.engine.functional.Functional

In [15]:
vgg_face_descriptor.inputs, vgg_face_descriptor.outputs

([<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'zero_padding2d_input')>],
 [<KerasTensor: shape=(None, 2622) dtype=float32 (created by layer 'flatten')>])

In [16]:
# Get embedding vector for first image in the metadata using the pre-trained model
img_path = metadata[0].image_path()
img = load_image(img_path)

# Normalising pixel values from [0-255] to [0-1]: scale RGB values to interval [0,1]
img = (img / 255.).astype(np.float32)
img = cv2.resize(img, dsize = (224,224))
print(img.shape)

# Obtain embedding vector for an image
# Get the embedding vector for the above image using vgg_face_descriptor model and print the shape
embedding_vector = vgg_face_descriptor.predict(np.expand_dims(img, axis=0))[0]
print(embedding_vector.shape)

(224, 224, 3)
1/1 [==============================] - 3s 3s/step
(2622,)


In [17]:
embedding_vector[0], type(embedding_vector), type(embedding_vector[0])

(0.015672842, numpy.ndarray, numpy.float32)

In [18]:
embedding_vector[2], embedding_vector[98], embedding_vector[-2]

(0.0073238397, 0.0022011711, 0.0142918825)

In [50]:
total_images = len(metadata)

print('total_images :', total_images)

total_images : 5250


In [51]:
embeddings = np.zeros((metadata.shape[0], 2622))
for i, m in enumerate(metadata):
    img_path = metadata[i].image_path()
    img = load_image(img_path)
    img = (img / 255.).astype(np.float32)
    img = cv2.resize(img, dsize = (224,224))
    embedding_vector = vgg_face_descriptor.predict(np.expand_dims(img, axis=0),verbose=0)[0]
    embeddings[i]=embedding_vector

In [54]:
embeddings[5249]

array([ 2.05632485e-02, -3.10130417e-05,  1.21988868e-02, ...,
       -4.87567158e-03,  3.28135933e-03,  7.03019742e-03])

In [75]:
def distance(emb1, emb2, threshold=0.8):

    # Calculate the distance between the embeddings
    embedding_distance = emb1 - emb2

    # Calculate the L2 norm of the distance vector
    embedding_distance_norm = np.linalg.norm(embedding_distance)

    # Return 1 if the distance is less than the threshold, else 0
    return embedding_distance_norm if embedding_distance_norm < threshold else 0

In [76]:
def recognize(img_path):
  img = load_image(img_path)
  img = (img / 255.).astype(np.float32)
  img = cv2.resize(img, dsize = (224,224))
  embedding_vector = vgg_face_descriptor.predict(np.expand_dims(img, axis=0),verbose=0)[0]
  distances = []
  names = []
  for i in range(len(embeddings)):
    dist = distance(embedding_vector,embeddings[i])
    if dist > 0:
      distances.append(dist)
      names.append(metadata[i].name)
  if distances:
    min_dist = min(distances)
    return names[distances.index(min_dist)]

  return "No Match Found"

In [77]:
print(recognize('/content/amanda_crew.jpg'))

pins_Amanda Crew


In [29]:
model.save('fism.keras')

In [80]:
import pickle
with open('utils.pkl','wb') as f:
  pickle.dump([embeddings,metadata],f)

In [32]:
from keras.models import load_model
new_model = load_model('/content/fism.keras')

In [35]:
imgt = load_image(metadata[5000].image_path())
imgt = (imgt / 255.).astype(np.float32)
imgt = cv2.resize(imgt, dsize = (224,224))
et = vgg_face_descriptor.predict(np.expand_dims(imgt, axis=0),verbose=0)[0]

In [36]:
et

array([ 0.00609213,  0.0066778 ,  0.01688961, ..., -0.02192893,
        0.00516967,  0.01105241], dtype=float32)